In this notebook, we will be taking a different approach to looking at extinction. Here, we'll rescue a group of individuals that are at a critical density and look to see if genetic diversity can be recovered within this population. 

To model this, we'll assign a linear vector of SNPs to each individual. Each element in the vector is 0, 1, or 2 which corresponds to the number of mutant polymorphisms in the genome at that position. For example, if we have a G -> A mutation, we would put 0 for GG, 1 for GA or AG, or 2 for AA. This linear vector is based on a pair of binary vectors of the same length, which determines whether that SNP is the mutant on a specific homolog. The 0, 1, 2 vector is obviously calculated then by element-wise adding the two vectors.

Thus, a population can be modeled as a matrix. The model will randomly generate a matrix or take in an input matrix from the user. What it needs to take in, however, is either a distance array or an LD array. For $n$ SNPs, this means an $n-1$ length vector of either base pair distances or LD values between the SNPs. That is, the 0th element of the linkage array corresponds to the linkage between the 0th and 1st SNPs. 

New individuals are created by choosing two individuals. For each parent, we 50/50 start with one of the two homologs, then pick the next mutant based on the provided distance/linkage disequilibrium. This goes until the end of the vector. The two vectors are then combined to make the new individual.

We assume no other mutations, and that all parents die after each reproduction period, so all new individuals in the next generation are good. We also assume linear growth from provided critical number to provided thriving population number, but a critical assumption here is that one new individual is created per generation. This reflects (often) low reproduction rates with many endangered species, especially larger animals and mammals. 

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def simulate_genetic_recovery(n_init,
                              n_thriving,
                              num_snps,
                              ld_array = None,
                              dist_array = None,
                              snp_mats = None):
    """
    Simulates a model of genetic recovery, where we start with a population of n_init and add one new individual
    until we reach n_thriving. The new individual's genotype is determined with probabilities based on the 
    provided LD or distance array. 

    Inputs:
    n_init: int, initial population size
    n_thriving: int, target population size
    num_snps: int, number of SNPs
    ld_array: 1D array of length num_snps-1, optional. This array contains the pairwise LD values between the SNPs.
    dist_array: 1D array of length num_snps-1, optional. This array contains the pairwise distances between the SNPs in megabases, or centimorgans.
        Note that one of ld_array or dist_array must be provided, and if both are, ld_array will be used.
    snp_mats: Array of 2D-arrays, optional. The number of 2D arrays must be equal to n_init. 
        Each 2D array must have shape (2, num_snps); each row is binary and contains the genotype of the individual on the paternal
        or maternal chromosome. It doesn't particularly matter which is which, but the first row is assumed to be the paternal chromosome.
        If this is not provided, the function will generate random genotypes for the initial population.

    Returns:
    snp_mats: Array of 2D-arrays. The number of 2D arrays is equal to n_thriving. 
        Each 2D array has shape (2, num_snps); each row is binary and contains the genotype of the individual on the paternal
        or maternal chromosome. It doesn't particularly matter which is which, but the first row is assumed to be the paternal chromosome.
    """

    #first: one of ld_array or dist_array must be provided
    if ld_array is None and dist_array is None:
        raise ValueError("Either ld_array or dist_array must be provided")
    
    #second: if snp_mats is not provided, generate random genotypes for the initial population
    if snp_mats is None:
        snp_mats = generate_random_genos(n_init, num_snps)
    elif len(snp_mats) != n_init:
        raise ValueError("The number of provided SNP matrices must be equal to n_init")
    elif snp_mats[0].shape[1] != num_snps:
        raise ValueError("The number of SNPs in the provided SNP matrices must be equal to num_snps")
    elif len(snp_mats[0]) != 2:
        raise ValueError("Each SNP matrix must have shape (2, num_snps)")
    
    probs = None
    #third: if ld_array is provided, check that it has the correct length; if it does, convert to probabilities
    if ld_array is not None:
        if len(ld_array) != num_snps - 1:
            raise ValueError("The length of ld_array must be equal to num_snps - 1")
        else:
            print("TODO")

    #fourth: if dist_array is provided, check that it has the correct length; if it does, convert to probabilities
    if dist_array is not None:
        if len(dist_array) != num_snps - 1:
            raise ValueError("The length of dist_array must be equal to num_snps - 1")
        else:
            print("TODO")

    for i in range(n_init, n_thriving):
        next_gen = []
        for j in range(i + 1):
            #pick two random individuals from the population
            ind1 = np.random.randint(0, len(snp_mats))
            ind2 = np.random.randint(0, len(snp_mats))
            while ind1 == ind2:
                ind2 = np.random.randint(0, len(snp_mats)) #ensure they are different
        
            #using these, reproduce a new individual
            new_ind = generate_new_ind(snp_mats[ind1], snp_mats[ind2], probs)

            #add the new individual to the population
            next_gen.append(new_ind)

        snp_mats = next_gen
        #print the current population size
        print("Current population size: ", len(snp_mats))

    return None

#helper function: generate random genotypes, given n_init and num_snps
def generate_random_genos(n_init, num_snps):
    snp_mats = []
    for i in range(n_init):
        snp_mat = np.random.randint(0, 2, size=(2, num_snps))
        snp_mats.append(snp_mat)
    return snp_mats

#helper function: given an LD array, convert to probabilities
#for simplicity: anything above 0.5 is kept as the original, and anything below is raised to 0.5
def convert_ld_to_probs(ld_array):
    probs = np.zeros(len(ld_array))
    for i in range(len(ld_array)):
        if ld_array[i] > 0.5:
            probs[i] = ld_array[i]
        else:
            probs[i] = 0.5
    return probs

#helper function: given a distance array, convert to probabilities
#for simplicity: anything distance x below 50 is converted to 1 - (x/100), and anything above is converted to 0.5
def convert_dist_to_probs(dist_array):
    probs = np.zeros(len(dist_array))
    for i in range(len(dist_array)):
        if dist_array[i] < 50:
            probs[i] = 1 - (dist_array[i]/100)
        else:
            probs[i] = 0.5
    return probs

#helper function: given two individuals' SNP matrices and a probability array, generate a new individual
def generate_new_ind(ind1, ind2, probs):
    num_snps = ind1.shape[1]
    new_ind = np.zeros((2, num_snps))

    #first, the paternal chromosome from ind1
    current_chrom = np.random.randint(0, 2) #0 or 1
    for i in range(num_snps):
        new_ind[0, i] = ind1[current_chrom, i]

        #flip a coin to determine whether to use the other chromosome
        if i == num_snps - 1:
            continue

        flip = np.random.rand()
        if flip > probs[i]:
            current_chrom = 1 - current_chrom

    #second, the maternal chromosome from ind2
    current_chrom = np.random.randint(0, 2) #0 or 1
    for i in range(num_snps):
        new_ind[1, i] = ind2[current_chrom, i]

        #flip a coin to determine whether to use the other chromosome
        if i == num_snps - 1:
            continue

        flip = np.random.rand()
        if flip > probs[i]:
            current_chrom = 1 - current_chrom

    return new_ind


In [5]:
generate_random_genos(10, 5)

[array([[0, 1, 0, 1, 1],
        [1, 1, 1, 0, 1]]),
 array([[1, 1, 0, 1, 1],
        [1, 1, 1, 0, 1]]),
 array([[1, 1, 0, 0, 1],
        [0, 1, 1, 0, 0]]),
 array([[0, 0, 0, 0, 1],
        [0, 1, 1, 1, 1]]),
 array([[1, 0, 0, 0, 1],
        [0, 1, 0, 0, 0]]),
 array([[1, 1, 1, 1, 0],
        [0, 0, 0, 1, 1]]),
 array([[0, 1, 1, 0, 1],
        [0, 1, 1, 0, 1]]),
 array([[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0]]),
 array([[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1]]),
 array([[1, 0, 0, 0, 1],
        [0, 0, 0, 0, 0]])]